En este jupyter voy a buscar una lista de ciudades, inicialmente utilizaré una lista de 300 ciudades con mayor pib del mundo de wikipedia, pero tengo que aportarles sus datos, para ello probaré distintos datasets o scrapearé paginas web e iré filtrando lo que me interese

In [1]:
import pandas as pd
from selenium import webdriver
from dotenv import load_dotenv
from googlemaps import convert
import googlemaps
import os
import requests
import json


In [ ]:
ciudades= pd.read_csv ()

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [5]:
url="https://es.wikipedia.org/wiki/Anexo:Ciudades_por_PIB"
driver.get(url)

In [6]:
tablaciudades= driver.find_element_by_xpath ("/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody")

In [7]:
listacitis=[]
for dato in tablaciudades.find_elements_by_tag_name("tr"):
    fila=[d for d in dato.find_elements_by_tag_name ("td")]
    if len (fila)>1 :
        dia= {
            "id": fila[0].text,
            "ciudad": fila[1].text,
            "pais": fila[2].text,
            "PIB": fila [3].text,
            "PIBpc": fila [4].text,
            "Continente": fila [5].text
             }
        listacitis.append (dia)

In [8]:
ciudades = pd.DataFrame(listacitis)
ciudades

,id,ciudad,pais,PIB,PIBpc,Continente
0,1,Tokio,Japón,1 617 000,68 776,Asia
1,2,Nueva York,Estados Unidos,1 403 463,69 915,América
2,3,Los Ángeles,Estados Unidos,860 452,65 082,América
3,4,París,Francia,845 500,57 241,Europa
4,5,Seúl,Corea del Sur,845 000,34 355,Asia
...,...,...,...,...,...,...
295,296,El Paso,Estados Unidos,32 659,38 737,América
296,297,Edimburgo,Reino Unido,32 497,58 437,Europa
297,298,Winnipeg,Canadá,32 478,41 719,América
298,299,Ciudad Juarez,México,32 400,12 434,América


In [8]:
ciudades.to_csv ("../Data/ciudades.csv")

### Sobre las variables, una idea inicial sería la siguiente:

#### 1. Ciudad
    - Array con nombres en varios formatos ( español, inglés, original, mayúsculas, minúsculas...)
#### 2. País
    - Array con nombre en varios formatos
#### 3. Estado/comunidad/región
    - Array con nombre en varios formatos
#### 4. Id

#### 5. Código (ES)


#### 6. Económicas:
    * PIB
    * Salario medio
    * Tasa de ocupación
    * Precio vida (relacion con salario medio)
        - Transporte
        - Vivienda
        - Cerveza
        - Big mac
    * Tasa pobreza
    
#### 7. Demo/geográficas:
    * Población ( sacar densidad)
        - Segmentar ( sexo, edad, religion...)
    * Esperanza de vida
    * Tamaño 
    * Clima
        - dias de lluvia
        - dias de sol
        - temperatura media
        
#### 8. Socioculturales:
    * Forma de vida:
        - Principales formas de ocio:
            - Array
        - Número de restaurantes/bares ( y densidad)
        - Número de tratros
        - Conciertos en el último año
        - Deportes más seguidos
            - Array
     * Idiomas
          - Array con cooficiales
          - % de habla inglesa
          - otros
     * educación:
         - Número de colegios
         - Número de universidades 
         - % de poblacion con estudios superiores
         
          
     
       
    

Una vez con una lista básica, vamos a buscar las variables.
Empezaré con un dataset de Kaggle, en el que aparecen "todas las ciudades", y vamos a ver si nos sirve

In [9]:
df= pd.read_csv ("../Data/worldcitiespop.csv")

C:\Users\asiok\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173958 entries, 0 to 3173957
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Country     object 
 1   City        object 
 2   AccentCity  object 
 3   Region      object 
 4   Population  float64
 5   Latitude    float64
 6   Longitude   float64
dtypes: float64(3), object(4)
memory usage: 169.5+ MB


In [11]:
df.head(10)

,Country,City,AccentCity,Region,Population,Latitude,Longitude
0,ad,aixas,Aixàs,6,NaN,42.483333,1.466667
1,ad,aixirivali,Aixirivali,6,NaN,42.466667,1.500000
2,ad,aixirivall,Aixirivall,6,NaN,42.466667,1.500000
3,ad,aixirvall,Aixirvall,6,NaN,42.466667,1.500000
4,ad,aixovall,Aixovall,6,NaN,42.466667,1.483333
5,ad,andorra,Andorra,7,NaN,42.500000,1.516667
6,ad,andorra la vella,Andorra la Vella,7,20430.0,42.500000,1.516667
7,ad,andorra-vieille,Andorra-Vieille,7,NaN,42.500000,1.516667
8,ad,andorre,Andorre,7,NaN,42.500000,1.516667
9,ad,andorre-la-vieille,Andorre-la-Vieille,7,NaN,42.500000,1.516667


Vamos a cambiar a mayúscula la primera letra de la ciudad, en nuestro primer data está así. Probablemente tenga que pasar todo a unicode

In [12]:
df.City=df.City.str.capitalize()

In [13]:
df.isnull().sum()

Country             0
City                6
AccentCity          0
Region              8
Population    3125978
Latitude            0
Longitude           0
dtype: int64

De este dataset podemos guardarnos la población, la región, y sus coordenadas. Primero eliminaré los que no me interesan

In [14]:
df.dropna(inplace=True)

In [15]:
df.Population.min()

7.0

In [16]:
dfordenado= df.sort_values ("Population", ascending=False)

In [17]:
dfordenado[:500]

,Country,City,AccentCity,Region,Population,Latitude,Longitude
1544449,jp,Tokyo,Tokyo,40,31480498.0,35.685000,139.751389
570824,cn,Shanghai,Shanghai,23,14608512.0,31.045556,121.399722
1327914,in,Bombay,Bombay,16,12692717.0,18.975000,72.825833
2200161,pk,Karachi,Karachi,05,11627378.0,24.905600,67.082200
1331162,in,Delhi,Delhi,7,10928270.0,28.666667,77.216667
...,...,...,...,...,...,...,...
2367739,ru,Barnaul,Barnaul,04,599586.0,53.360556,83.763611
2993640,us,Charlotte,Charlotte,NC,598351.0,35.226944,-80.843333
3027061,us,El paso,El Paso,TX,597181.0,31.758611,-106.486389
529698,cn,Luancheng,Luancheng,10,597130.0,37.879167,114.651667


In [18]:
listavalida=ciudades.ciudad.unique()

In [19]:
len (listavalida) # Revisar duplicados

298

In [20]:
df.City.isin (listavalida).sum() # Esto quiere decir que en nuestro df hay valores de listavalida que se duplican

327

Antes de continuar por este camino, creo que tengo que replantearlo, es inviable encontrar los datos que necesito para 300 ciudades en 4 días ( 40 min para buscar población, y en realidad no la tengo ni unida a nuestro data original). Los tendría que buscar uno a uno y cargarlos en la base de datos uno a uno. Además que la información existente en internet es muy limitada una vez sales de la zona "occidental". Encontrar estos datos me llevaría demasiado tiempo.


Puedo continuar con algo similar, haciendo un recomendador de ciudades españolas para ir de vacaciones, tendría que modificar las variables, eliminando las económicas y aumentando las de forma de ocio, y en lugar de centrarme en visualizaciones, pedirle al usuario unos inputs en los que luego yo pondere de alguna manera para darle una recomendación.

Una vez hecha una lista con 25-30 ciudades españolas, la idea es aumentar el rango por lo menos a Europa. Pero creo que es importante empezar con pocas ciudades para que, en el peor de los casos, pueda transcribir la base de datos a mano.

Quesaría algo así:
(collection=ciudad)
#### 1. Ciudad
    - Array con nombres en varios formatos ( español, inglés, unicode y original en su caso) (transcribir)
#### 2. País
    - Array con nombre en varios formatos (transcribir)
#### 3. Ubicación
    - Coordenadas google place

#### 6. Presupuesto de viaje:
    - Precio medio noche hotel (transcribir)
    - Precio alquiler en temporada alta (transcribir)
    - Precio restaurante (transcribir) (tripadvisor)
    - Precio cerveza medio (transcribir)
    - Precio transporte origen-destino (api de google place)

    
#### 7. Carácteristicas ( las iré definiendo según busque información):
    * Costa ( transcribir)
    - Museos, hitos culturales ( api de fourthsquare o gplace)
    - turismo joven, solteros, familias...
    - formas de ocio ( las que me encuentre por interner y luego localizadas mediante api) 
    ej:
        - surf (buscar api)
            - numero de playas habilitadas 
            - numero de escuelas/alquiler materials
        - senderismo
            - numero de rutas
            - ...
        - gastronomia:
            - numero de restaurantes
            - variedad de cocinas
            
            
            
inputs ( rango edad, sexo, preferencias y cómo de importante son de 1 a 5, presupuesto, tiempo de viaje)
algoritmo de puntuación
output

Objetivo jueves:
- tener al menos 1 ciudad con todos los datos)

Objetivo viernes:
- tener todas las ciudades, empezar con la api "basica", definir los post y gets básicos que no comparen)

Objetivo sabado:
- terminar api y empezar con el algoritmo.

Objetivo domingo:
- terminar algoritmo y meterlo en la api


#### Extras: ampliar database, recomendar varias ciudades y comparar, buscar hotel, meter fotos de las ciudades...

In [3]:
url="https://www.enterat.com/ocio/ciudades-mas-visitadas-espana.php"
driver.get(url)

In [4]:
tablaciudades= driver.find_element_by_xpath ("/html/body/div[2]/div[2]/div[1]/div[6]/table[2]/tbody")

In [5]:
listacitis=[] #funcion, recibe un xpath y una lista(id, ciudad...)
for dato in tablaciudades.find_elements_by_tag_name("tr"):
    fila=[d for d in dato.find_elements_by_tag_name ("td")]
    if len (fila)>1 :
        dia= {
            "id": fila[0].text,
            "ciudad": fila[1].text,
            "Viajeros": fila[2].text
             }
        listacitis.append (dia)

Empezamos con las 25 primeras, y si nos da tiempo a aumentar, siempre tenemos esta lista cómo guía

In [6]:
ciudades= pd.DataFrame (listacitis[:26])

In [7]:
ciudades.drop([0],axis=0, inplace=True)


Todo lo que vaya añadiendo a mi dataframe y quiera guardar luego en mongo lo iré guardando como arrays

In [9]:
ciudades.ciudad.to_csv ("../Data/nombreciudades.csv")

Ahora vamos a utilizar google place para obtener toda la información que nos pueda ser útil

In [9]:
ciudades

,id,ciudad,Viajeros
1,1,MADRID,9.963.195
2,2,BARCELONA,8.520.415
3,3,SEVILLA,2.811.351
4,4,PALMA DE MALLORCA,2.372.933
5,5,BENIDORM,2.150.417
6,6,GRANADA,2.001.464
7,7,VALENCIA,1.882.918
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975
9,9,CALVIÀ,1.540.863
10,10,ADEJE,1.476.042


In [10]:
load_dotenv()

tokg = os.getenv("tokg")

In [11]:
gmaps = googlemaps.Client(key= tokg )

In [74]:
gmaps.places_nearby (
        location= "madrid",
        type = ["Hotel"],
        radius= 1000
         )

ApiError: INVALID_REQUEST

Tan facil no iba a ser, vamos primero a ver como conseguimos las coordenadas, que es lo que pide la api para "location", las voy a guardar por separado y unidas, en previsión de cómo me las pidan las apis.

In [13]:
coords=gmaps.geocode( address="MADRID")

In [14]:
coords

[{'address_components': [{'long_name': 'Madrid',
    'short_name': 'Madrid',
    'types': ['locality', 'political']},
   {'long_name': 'Madrid',
    'short_name': 'M',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Community of Madrid',
    'short_name': 'MD',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Spain',
    'short_name': 'ES',
    'types': ['country', 'political']}],
  'formatted_address': 'Madrid, Spain',
  'geometry': {'bounds': {'northeast': {'lat': 40.5638447, 'lng': -3.5249115},
    'southwest': {'lat': 40.3120639, 'lng': -3.8341618}},
   'location': {'lat': 40.4167754, 'lng': -3.7037902},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 40.5638447, 'lng': -3.5249115},
    'southwest': {'lat': 40.3120639, 'lng': -3.8341618}}},
  'place_id': 'ChIJgTwKgJcpQg0RaSKMYcHeNsQ',
  'types': ['locality', 'political']}]

In [15]:
coords[0].keys()

dict_keys(['address_components', 'formatted_address', 'geometry', 'place_id', 'types'])

In [141]:
a=coords[0]["geometry"]["location"]

In [190]:
c= str (a.values())

In [287]:
gmaps.geocode( address="barcelona", region="ES")

[{'address_components': [{'long_name': 'Barcelona',
    'short_name': 'Barcelona',
    'types': ['locality', 'political']},
   {'long_name': 'Barcelona',
    'short_name': 'B',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Catalonia',
    'short_name': 'CT',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Spain',
    'short_name': 'ES',
    'types': ['country', 'political']}],
  'formatted_address': 'Barcelona, Spain',
  'geometry': {'bounds': {'northeast': {'lat': 41.4695761, 'lng': 2.2280099},
    'southwest': {'lat': 41.320004, 'lng': 2.0695258}},
   'location': {'lat': 41.3850639, 'lng': 2.1734035},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 41.4695761, 'lng': 2.2280099},
    'southwest': {'lat': 41.320004, 'lng': 2.0695258}}},
  'place_id': 'ChIJ5TCOcRaYpBIRCmZHTz37sEQ',
  'types': ['locality', 'political']}]

In [23]:
def coordenadas (df,columna):
    """
    Función que recibe como parámetro un DataFrame y una columna donde se encuentren nombres de ciudades, y dá como salida el
    DataFrame con tres columnas nuevas. Dos con con la latitud y longitud separadas, y otra más con un diccionario con ambas
    
    La geocidificación se hace mediante la api de googleplace y la libreria gmaps, por lo que es necesario disponer de token
    y ya haberlo incluido en "gmaps"
    
    No recomendable con dataframes demasiado extensos por el número de request
    """
    latitudes=[]
    longitudes=[]
    coordenadas=[]
    coordenadas2=[]
    for i in df[columna]:
        
        coords = gmaps.geocode( address=i, region="ES")
        longitudes.append (coords[0]["geometry"]["location"]["lng"])
        latitudes.append (coords[0]["geometry"]["location"]["lat"])
        coordenadas.append (coords[0]["geometry"]["location"])
        coordenadas2.append (coords[0]["geometry"]["location"].values())
    df["Lon"]= longitudes
    df["Lat"]= latitudes
    df["coords"] = coordenadas
    df["coords2"] = coordenadas2
    return ciudades

In [24]:
coordenadas(ciudades,"ciudad")

,id,ciudad,Viajeros,Lon,Lat,coords,coords2
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","(40.4167754, -3.7037902)"
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","(41.3850639, 2.1734035)"
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","(37.3890924, -5.9844589)"
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","(39.5696005, 2.6501603)"
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","(38.5410566, -0.1224937)"
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","(37.1773363, -3.5985571)"
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","(39.4699075, -0.3762881)"
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","(27.9249459, -15.5730295)"
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","(39.5653483, 2.5033683)"
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","(28.1220604, -16.7338211)"


In [425]:
ciudades.Lon.to_csv ("../Data/Longitudciudades.csv")

In [426]:
ciudades.Lat.to_csv ("../Data/Latitudciudades.csv")

In [427]:
ciudades.coords.to_csv ("../Data/coordenadasciudades.csv")

In [430]:
ciudades.head()

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nhoteles,listahoteles,npubs,lpubs
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Senator Gran Via 70 Spa Hotel 4*', '...",4,"{'name': 'Destellos Pub Disco', 'latitud': 40...."
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Hotel Medinaceli', 'latitud': 41.378...",0,{}
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Hotel Palacio Villapanes', 'latitud'...",0,{}
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Café Gran Hotel', 'latitud': 39.5725...",0,{}
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Hotel Agir Kamer 211', 'latitud': 38...",3,"{'name': 'La Posada Disco Pub', 'latitud': 38...."


In [431]:
ciudades.coords2.to_csv ("../Data/coordenadas2ciudades.csv")

In [432]:
ciudades.nhoteles.to_csv ("../Data/nhoteles.csv")

In [433]:
ciudades.listahoteles.to_csv ("../Data/lhoteles.csv")

In [434]:
ciudades.npubs.to_csv ("../Data/npubs.csv")

In [435]:
ciudades.lpubs.to_csv ("../Data/lpubs.csv")

Ahora lo que vamos a hacer es ver una exploración de la api de gmaps para los hoteles, ver qué información podemos sacar, como guardarla y terminaremos este apartado preparando algún bucle para agregarla a nuestro dataframe.

In [56]:
madrid= ciudades.coords[1]

In [33]:
madrid

{'lat': 40.4167754, 'lng': -3.7037902}

In [57]:
a=gmaps.places_nearby (
        location= madrid,
        type = ["Hotel"],
        radius= 2000
         )

In [62]:
a["results"] # geometry': {'location': {'lat': 40.4175032, 'lng': -3.7033915},
             #  name': 'Madrid',

[{'geometry': {'location': {'lat': 40.4167754, 'lng': -3.7037902},
   'viewport': {'northeast': {'lat': 40.56384473341083,
     'lng': -3.524911528167741},
    'southwest': {'lat': 40.31206394740597, 'lng': -3.834161809872056}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'name': 'Madrid',
  'photos': [{'height': 4000,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115573911546909190179">Manuel Colomo San Valeriano</a>'],
    'photo_reference': 'ATtYBwLuNWgbAaWoKvvhUxmj3Nc2b-7wt3e_iqQXBEY5d6AIKk8J6HlssrGKvzo_hcIMB-MFG2qCEXtnMidpvp9bre5G1b4U2oQenwZp6DQ44oCjroREKPfzmIA8Gg8IsRR1ZnO-u14AJ2NghxXHPQbBn2f2S1Ox0CgmHrW7Gt3CXV2uUsCK',
    'width': 3000}],
  'place_id': 'ChIJgTwKgJcpQg0RaSKMYcHeNsQ',
  'reference': 'ChIJgTwKgJcpQg0RaSKMYcHeNsQ',
  'scope': 'GOOGLE',
  'types': ['locality', 'political'],
  'vicinity': 'Madrid'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 40.4175032, 'lng': -3.70339

In [96]:
a3=gmaps.places_nearby (page_token= a5 )

In [97]:
len (a3["results"])

20

Con esto añadiremos la columna nº de hoteles en 2km radio centro

In [109]:
def numero_hoteles(df, columna):
    numerohoteles=[]
    for i in df[columna]:
        page1 = gmaps.places_nearby (
        location= i,
        type = ["Hotel"],
        radius= 2000
         )
       
        numerohoteles.append (n)
    print (numerohoteles)
            
               
            
   

In [110]:
numero_hoteles (ciudades, "coords")

ATtYBwJnt_bg2CWqhD4LwdjkKLDVZacdecrtfu9SGfyZ8RX6RhgX7Ixi2u21PuIO-6jtRTbgrAotcFPNryS_yBXA6sl4ZcKAguw7yDsZLEweXlXhgd5GzPpUVmbDoS0CYmycPJQncoO_fjHn_VnSolIaxaspWP17si6oleK9h4gU36ex_n1OJjpgUkrSq4Ub92dcrHCqH8TUSyQIX4B9uH_nunyMs9JcaJmU-LkKf1dDLOs3rJnf9PPHaogpKU3I8-HPwynTxankJ72amxWlWn01Nh5WDwhstnyfilCRmMLk-K-R2DuSDd_4BDXN-bloWqDLVjmlFMJqTPNKimg7KWTN5WbliUDrLERnRl3Hi4n58v0lwJTYQDVB6Spp-f9XbGylGISDapAAQGmNj5eGsz7uD_LTtR-Ln1yMgGoJaD049bSShe5x7dGgcQ


ApiError: INVALID_REQUEST

In [115]:
token1="ATtYBwJnt_bg2CWqhD4LwdjkKLDVZacdecrtfu9SGfyZ8RX6RhgX7Ixi2u21PuIO-6jtRTbgrAotcFPNryS_yBXA6sl4ZcKAguw7yDsZLEweXlXhgd5GzPpUVmbDoS0CYmycPJQncoO_fjHn_VnSolIaxaspWP17si6oleK9h4gU36ex_n1OJjpgUkrSq4Ub92dcrHCqH8TUSyQIX4B9uH_nunyMs9JcaJmU-LkKf1dDLOs3rJnf9PPHaogpKU3I8-HPwynTxankJ72amxWlWn01Nh5WDwhstnyfilCRmMLk-K-R2DuSDd_4BDXN-bloWqDLVjmlFMJqTPNKimg7KWTN5WbliUDrLERnRl3Hi4n58v0lwJTYQDVB6Spp-f9XbGylGISDapAAQGmNj5eGsz7uD_LTtR-Ln1yMgGoJaD049bSShe5x7dGgcQ"

In [116]:
gmaps.places_nearby ( page_token = token1 )

{'html_attributions': [],
 'next_page_token': 'ATtYBwJjQrnyZnPquotHzzp9AAknotfmx3LeVT-Hl1dSBYTOfCjiblV4PkTIJqg_Uf-fCCE4-Bx4YAwRd6UdsLgWote_qVo9hqHN8iJmimLrxjxw26T7t74sjehzKeOzXInemRewhn3O81siOo5f-eoHVfM3BkfEVH2ez9enjYvHBp3Fms3CmEtprl6ckqXiTha3okvkhvsjDllmP1L1xNHjMDpumvdW44NV_06Zz6N788mecepgar886lK2-Ddw7TvGZmq1DTm2Cx-n_0oZYecADe3KpFiGtlkKhZ1JTeh9Yedu86xCiIvsxSA_fUnR-O8nVsiDP0p82pK2-gf16mvCdiihM838-JOdFOIzVcaOQ48rQuRwboz9k4gc2PNfRlgJ_1rhCluCu8A0wtSdm-JLSzM9QhzMq_219Hs2s6Xwi-Zrig8bJhobyNvqSJyRAdYWZi4aLNEUp46Yu0MeJcHaWKLMxnaHMMKBt1OfrLYXWIsKdPuzpFco7rfDKruKW8Y0OJ9ap96TCNF_aMmqUy-e0zKDn0aSY7z22Vtsosnq0pfbAZUQurVKzpw4fyCznstv0xx4oezmGFkSsJtKlOo9Fovk-H4tMWbD0nY2zWPM0xxAlNQ5Etu6TLNiYZtrsh8XD3MEcfKeVXo',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 40.4131012, 'lng': -3.7086039},
    'viewport': {'northeast': {'lat': 40.4144345802915,
      'lng': -3.707213969708498},
     'southwest': {'lat': 40.4117366197085, 'lng': -3.709911930291502}}},
   'icon': 'htt

In [131]:
load_dotenv()

True

In [12]:
tokf1 = os.getenv("tokf1")
tokf2 = os.getenv("tokf2")
tokf3 = os.getenv ("tokf3")

In [134]:
print (tokf3)

KAOV2DGRYBHLVV2IQ3ODKOQEPKAMGXADQI2H3VCFV4HZELDW


In [13]:
import foursquare

In [14]:
client = foursquare.Foursquare( access_token= tokf3 )

In [213]:
madrid=(ciudades.coords2[1])

In [177]:
madrid.values()

AttributeError: 'dict_values' object has no attribute 'values'

In [32]:
hotelesmadrid=client.venues.search(params={'query': 'hotel', 'll': "40.4167754, -3.7037902", "radius": 2000, "limit": 50})

In [ ]:
def pasando_a_string (item):
        string_= str (item)
        recortando= string_[13:34]
        return recortando

In [26]:
ciudades.coords2 = ciudades.coords2.apply (pasando_a_string)

In [33]:
lista=hotelesmadrid["venues"]

In [293]:
hoteles(ciudades, "coords2")

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nhoteles,listahoteles
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Senator Gran Via 70 Spa Hotel 4*', '..."
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Hotel Medinaceli', 'latitud': 41.378..."
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Hotel Palacio Villapanes', 'latitud'..."
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Café Gran Hotel', 'latitud': 39.5725..."
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Hotel Agir Kamer 211', 'latitud': 38..."
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","37.1773363, -3.598557",50,"{'name': 'Hotel Palacio De Mariana Pineda', 'l..."
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","39.4699075, -0.376288",50,"{'name': 'Hotel SH Inglès', 'latitud': 39.4729..."
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","27.9249459, -15.57302",2,"{'name': 'Mirador Hotel Las Tirajanas', 'latit..."
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","39.5653483, 2.5033683",1,"{'name': 'Finca Can Carro', 'latitud': 39.5563..."
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","28.1220604, -16.73382",8,"{'name': 'Pool @ Hotel Suite Villa Maria', 'la..."


In [31]:
restaurantesmadrid=client.venues.search(params={'query': 'Restaurante', 'll': madrid, "radius": 2000, "limit": 50})

NameError: name 'madrid' is not defined

In [396]:
lista=restaurantesmadrid["venues"]

In [422]:
lista[31]["categories"][0]["id"]

'4bf58dd8d48988d150941735'

In [437]:
lala=[]
for i in range (30):
    lala.append (lista [i]["categories"][0]["id"])
    

IndexError: list index out of range

In [401]:
for i in range (len(lista)-48):
    print (i)
  

0
1


In [53]:
def rellenando_datasetfsq (df, columna, query, rango, nueva, nueva2):
    """
    blablabla
    """
    cantidad = []
    sublista = []
    
    for ciudad in df[columna] :
        
        request = client.venues.search(params = {'query': query , 'll': ciudad, "radius": rango, "limit": 50})
        lista = request["venues"]
        cantidad.append (len (lista))
        
        dic = {}
        for i in range ( len (lista)-1) :
            
            dic["name"] = lista[i]["name"]
            dic["latitud"] = lista[i]["location"]["lat"]
            dic["longitud"] = lista[i]["location"]["lng"]
            
        sublista.append (dic)
        
    df[nueva]=cantidad
    df[nueva2] = sublista
    return df

In [379]:
len (lista) -1

8

In [54]:
rellenando_datasetfsq (ciudades, "coords2", ["Restaurante", "Restaurant"], 2000, "nrestaurantes", "lrestaurantes")

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nrestaurantes,lrestaurantes
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Restaurante Pozo Real', 'latitud': 4..."
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Restaurante Chino Internacional', 'l..."
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Restaurante Trashumante', 'latitud':..."
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Restaurante Ca'n Julià', 'latitud': ..."
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Restaurante Hotel Brisa', 'latitud':..."
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","37.1773363, -3.598557",50,{'name': 'Restaurante Asador El Rincón de San ...
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","39.4699075, -0.376288",50,"{'name': 'Restaurante Premium Apartaments', 'l..."
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","27.9249459, -15.57302",3,"{'name': 'Restaurante La Cueva', 'latitud': 27..."
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","39.5653483, 2.5033683",2,{'name': 'Restaurante Café Ajuntament de Calvi...
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","28.1220604, -16.73382",19,"{'name': 'Restaurant', 'latitud': 28.125280380..."


In [342]:
rellenando_dataset (ciudades, "coords2", "4bf58dd8d48988d1e2941735", 3000, "nplayas", "lplayas")

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nhoteles,listahoteles,nrestaurantes,lrestaurantes,nplayas,lplayas
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Senator Gran Via 70 Spa Hotel 4*', '...",12,"{'name': 'Restaurante Siam', 'latitud': 40.424...",0,{}
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Hotel Medinaceli', 'latitud': 41.378...",4,"{'name': 'Restaurante Sailor', 'latitud': 41.3...",0,{}
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Hotel Palacio Villapanes', 'latitud'...",5,"{'name': 'Restaurante San Marcos', 'latitud': ...",0,{}
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Café Gran Hotel', 'latitud': 39.5725...",9,"{'name': 'Es Baluard Restaurant & Lounge', 'la...",0,{}
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Hotel Agir Kamer 211', 'latitud': 38...",3,"{'name': 'Restaurant espejos', 'latitud': 38.5...",0,{}
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","37.1773363, -3.598557",50,"{'name': 'Hotel Palacio De Mariana Pineda', 'l...",8,"{'name': 'Fluxus Restaurante Sonoro', 'latitud...",0,{}
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","39.4699075, -0.376288",50,"{'name': 'Hotel SH Inglès', 'latitud': 39.4729...",9,{'name': 'Cervecería Restaurante Sierra Aitana...,0,{}
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","27.9249459, -15.57302",2,"{'name': 'Mirador Hotel Las Tirajanas', 'latit...",0,{},0,{}
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","39.5653483, 2.5033683",1,"{'name': 'Finca Can Carro', 'latitud': 39.5563...",0,{},0,{}
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","28.1220604, -16.73382",8,"{'name': 'Pool @ Hotel Suite Villa Maria', 'la...",0,{},0,{}


In [345]:
ciudades.drop (["nplayas", "lplayas"], axis=1, inplace=True) # estas las busco en gplaces

In [367]:
ciudades.drop (["nrestaurantes","lrestaurantes"], axis=1, inplace=True)

In [366]:
rellenando_dataset (ciudades, "coords2", ["Pub", "Disco", "Cocktail"], 2000, "npubs", "lpubs")

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nhoteles,listahoteles,nrestaurantes,lrestaurantes,npubs,lpubs
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Senator Gran Via 70 Spa Hotel 4*', '...",12,"{'name': 'Restaurante Siam', 'latitud': 40.424...",4,"{'name': 'Destellos Pub Disco', 'latitud': 40...."
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Hotel Medinaceli', 'latitud': 41.378...",4,"{'name': 'Restaurante Sailor', 'latitud': 41.3...",0,{}
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Hotel Palacio Villapanes', 'latitud'...",5,"{'name': 'Restaurante San Marcos', 'latitud': ...",0,{}
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Café Gran Hotel', 'latitud': 39.5725...",9,"{'name': 'Es Baluard Restaurant & Lounge', 'la...",0,{}
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Hotel Agir Kamer 211', 'latitud': 38...",3,"{'name': 'Restaurant espejos', 'latitud': 38.5...",3,"{'name': 'La Posada Disco Pub', 'latitud': 38...."
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","37.1773363, -3.598557",50,"{'name': 'Hotel Palacio De Mariana Pineda', 'l...",8,"{'name': 'Fluxus Restaurante Sonoro', 'latitud...",0,{}
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","39.4699075, -0.376288",50,"{'name': 'Hotel SH Inglès', 'latitud': 39.4729...",9,{'name': 'Cervecería Restaurante Sierra Aitana...,1,{}
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","27.9249459, -15.57302",2,"{'name': 'Mirador Hotel Las Tirajanas', 'latit...",0,{},0,{}
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","39.5653483, 2.5033683",1,"{'name': 'Finca Can Carro', 'latitud': 39.5563...",0,{},0,{}
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","28.1220604, -16.73382",8,"{'name': 'Pool @ Hotel Suite Villa Maria', 'la...",0,{},0,{}


In [364]:
playasmadrid=client.venues.search(params={'query': ["playa"], 'll': madrid, "radius": 2000, "limit": 50})

In [365]:
playasmadrid

{'venues': [{'id': '4c8094fff9b79c742b720d45',
   'name': 'Cava Baja 5 (La Playa)',
   'location': {'address': 'Cava Baja, 5',
    'lat': 40.413359511996894,
    'lng': -3.708349914681621,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.413359511996894,
      'lng': -3.708349914681621}],
    'distance': 542,
    'postalCode': '28005',
    'cc': 'ES',
    'city': 'Madrid',
    'state': 'Madrid',
    'country': 'España',
    'formattedAddress': ['Cava Baja, 5', '28005 Madrid Madrid']},
   'categories': [{'id': '4bf58dd8d48988d11e941735',
     'name': 'Cocktail Bar',
     'pluralName': 'Cocktail Bars',
     'shortName': 'Cocktail',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1620964715',
   'hasPerk': False},
  {'id': '4e45a73eb61c03d0cd015a44',
   'name': 'La Playa de Lavapies',
   'location': {'address': 'Argumosa 9',
    'lat': 40.40827840078169,
    'lng': -3.699

In [344]:
ciudades

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nhoteles,listahoteles,nrestaurantes,lrestaurantes,nplayas,lplayas
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Senator Gran Via 70 Spa Hotel 4*', '...",12,"{'name': 'Restaurante Siam', 'latitud': 40.424...",0,{}
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Hotel Medinaceli', 'latitud': 41.378...",4,"{'name': 'Restaurante Sailor', 'latitud': 41.3...",0,{}
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Hotel Palacio Villapanes', 'latitud'...",5,"{'name': 'Restaurante San Marcos', 'latitud': ...",0,{}
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Café Gran Hotel', 'latitud': 39.5725...",9,"{'name': 'Es Baluard Restaurant & Lounge', 'la...",0,{}
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Hotel Agir Kamer 211', 'latitud': 38...",3,"{'name': 'Restaurant espejos', 'latitud': 38.5...",0,{}
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","37.1773363, -3.598557",50,"{'name': 'Hotel Palacio De Mariana Pineda', 'l...",8,"{'name': 'Fluxus Restaurante Sonoro', 'latitud...",0,{}
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","39.4699075, -0.376288",50,"{'name': 'Hotel SH Inglès', 'latitud': 39.4729...",9,{'name': 'Cervecería Restaurante Sierra Aitana...,0,{}
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","27.9249459, -15.57302",2,"{'name': 'Mirador Hotel Las Tirajanas', 'latit...",0,{},0,{}
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","39.5653483, 2.5033683",1,"{'name': 'Finca Can Carro', 'latitud': 39.5563...",0,{},0,{}
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","28.1220604, -16.73382",8,"{'name': 'Pool @ Hotel Suite Villa Maria', 'la...",0,{},0,{}


In [424]:
ciudades

,id,ciudad,Viajeros,Lon,Lat,coords,coords2,nhoteles,listahoteles,npubs,lpubs
1,1,MADRID,9.963.195,-3.703790,40.416775,"{'lat': 40.4167754, 'lng': -3.7037902}","40.4167754, -3.703790",50,"{'name': 'Senator Gran Via 70 Spa Hotel 4*', '...",4,"{'name': 'Destellos Pub Disco', 'latitud': 40...."
2,2,BARCELONA,8.520.415,2.173404,41.385064,"{'lat': 41.3850639, 'lng': 2.1734035}","41.3850639, 2.1734035",50,"{'name': 'Hotel Medinaceli', 'latitud': 41.378...",0,{}
3,3,SEVILLA,2.811.351,-5.984459,37.389092,"{'lat': 37.3890924, 'lng': -5.9844589}","37.3890924, -5.984458",50,"{'name': 'Hotel Palacio Villapanes', 'latitud'...",0,{}
4,4,PALMA DE MALLORCA,2.372.933,2.650160,39.569600,"{'lat': 39.5696005, 'lng': 2.6501603}","39.5696005, 2.6501603",50,"{'name': 'Café Gran Hotel', 'latitud': 39.5725...",0,{}
5,5,BENIDORM,2.150.417,-0.122494,38.541057,"{'lat': 38.5410566, 'lng': -0.1224937}","38.5410566, -0.122493",50,"{'name': 'Hotel Agir Kamer 211', 'latitud': 38...",3,"{'name': 'La Posada Disco Pub', 'latitud': 38...."
6,6,GRANADA,2.001.464,-3.598557,37.177336,"{'lat': 37.1773363, 'lng': -3.5985571}","37.1773363, -3.598557",50,"{'name': 'Hotel Palacio De Mariana Pineda', 'l...",0,{}
7,7,VALENCIA,1.882.918,-0.376288,39.469907,"{'lat': 39.4699075, 'lng': -0.3762881}","39.4699075, -0.376288",50,"{'name': 'Hotel SH Inglès', 'latitud': 39.4729...",1,{}
8,8,SAN BARTOLOMÉ DE TIRAJANA,1.577.975,-15.573030,27.924946,"{'lat': 27.9249459, 'lng': -15.5730295}","27.9249459, -15.57302",2,"{'name': 'Mirador Hotel Las Tirajanas', 'latit...",0,{}
9,9,CALVIÀ,1.540.863,2.503368,39.565348,"{'lat': 39.5653483, 'lng': 2.5033683}","39.5653483, 2.5033683",1,"{'name': 'Finca Can Carro', 'latitud': 39.5563...",0,{}
10,10,ADEJE,1.476.042,-16.733821,28.122060,"{'lat': 28.1220604, 'lng': -16.7338211}","28.1220604, -16.73382",8,"{'name': 'Pool @ Hotel Suite Villa Maria', 'la...",0,{}
